In [73]:
import yfinance as yf
import pandas as pd
import os
import datetime
import pickle
import numpy as np

In [ ]:
# Some important functions
def save_obj(obj, name):
    with open('data/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
def getTickerDat(ticks):
tickerDat = {}
for t in ticks:
    print('{}% done'.format(round(100*(len(tickerDat)+1)/len(ticks)),6))
    stockDat = yf.download(t, 
                           start=insiderDat.TradeDate.min().strftime("%Y-%m-%d"), 
                           end=insiderDat.TradeDate.max().strftime("%Y-%m-%d"), 
                           progress=False)
    tickerDat[t] = stockDat
return tickerDat

In [76]:
insiderDat = pd.read_csv('data/openinsider_June.csv')

insiderDat = insiderDat.rename(columns={insiderDat.columns[1]: "FilingDate", 
                                        insiderDat.columns[2]: "TradeDate", 
                                        insiderDat.columns[4]: "CompanyName", 
                                        insiderDat.columns[5]: "InsiderName", 
                                        insiderDat.columns[7]: "TradeType",
                                        insiderDat.columns[11]: "DeltaOwn"},
                              errors="raise")
insiderDat.FilingDate = [datetime.datetime.strptime(r, '%m/%d/%y %H:%M') for r in insiderDat.FilingDate]
insiderDat.TradeDate = [datetime.datetime.strptime(r, '%m/%d/%y') for r in insiderDat.TradeDate]

insiderDat.Price = [float(p[1:].replace(',','')) for p in insiderDat.Price]
insiderDat.Qty = [float(q.replace(',','')) for q in insiderDat.Qty]
insiderDat.Owned = [float(o.replace(',','')) for o in insiderDat.Owned]
insiderDat = insiderDat.replace({'DeltaOwn': {'New': '999999'}})
insiderDat = insiderDat.replace({'DeltaOwn': {'>999%': '999999'}})
insiderDat.DeltaOwn = [float(d[:-1]) for d in insiderDat.DeltaOwn]
insiderDat = insiderDat.replace({'DeltaOwn': {'99999': np.nan}})
insiderDat.Value = [v.replace('(','') for v in insiderDat.Value]
insiderDat.Value = [v.replace(')','') for v in insiderDat.Value]
insiderDat.Value = [v.replace(',','') for v in insiderDat.Value]
insiderDat.Value = [float(v[1:]) for v in insiderDat.Value]

insiderDat.head()

,X,FilingDate,TradeDate,Ticker,CompanyName,InsiderName,Title,TradeType,Price,Qty,Owned,DeltaOwn,Value,1d,1w,1m,6m
0,NaN,2021-06-11 18:00:00,2021-06-11,IBP,"Installed Building Products, Inc.",Thomas Michael H,Dir,S - Sale,116.85,-2100.0,10093.0,-17.0,245378.0,NaN,NaN,NaN,NaN
1,NaN,2021-06-11 19:21:00,2021-06-11,DXC,Dxc Technology Co,Fernandez Raul J,Dir,P - Purchase,41.20,1820.0,31305.0,6.0,74975.0,NaN,NaN,NaN,NaN
2,NaN,2021-06-11 16:50:00,2021-06-11,REVG,"Rev Group, Inc.",Boettinger Stephen W.,GC,P - Purchase,15.64,1500.0,121560.0,1.0,23460.0,NaN,NaN,NaN,NaN
3,NaN,2021-06-11 19:43:00,2021-06-11,AMST,Amesite Inc.,Sastry Ann Marie,CEO,P - Purchase,3.73,15000.0,6241667.0,0.0,55950.0,NaN,NaN,NaN,NaN
4,NaN,2021-06-11 19:47:00,2021-06-11,TLYS,"Tilly'S, Inc.",Zeichner Bernard,Dir,S - Sale,16.24,-5000.0,46894.0,-10.0,81200.0,NaN,NaN,NaN,NaN


In [83]:
allTickers = insiderDat.Ticker.unique()
len(allTickers)

913

In [84]:
#historicDat = getTickerDat(allTickers)
#save_obj(historicDat, 'historicDat')

In [85]:
ghostTickers = ['SVMK', 'ISNS', 'GTXMQ', 'TCF', 'HOME', 'ARYA', 'ZANDZ', 'DILA', 'RACA', 'WPF', 'TRCH', 'USCR', 
                'KIN', 'STAY', 'HTZGQ']

insiderDat_filtered = insiderDat[insiderDat.Ticker.isin(ghostTickers)==False]
insiderDat_filtered.head()

,X,FilingDate,TradeDate,Ticker,CompanyName,InsiderName,Title,TradeType,Price,Qty,Owned,DeltaOwn,Value,1d,1w,1m,6m
0,NaN,2021-06-11 18:00:00,2021-06-11,IBP,"Installed Building Products, Inc.",Thomas Michael H,Dir,S - Sale,116.85,-2100.0,10093.0,-17.0,245378.0,NaN,NaN,NaN,NaN
1,NaN,2021-06-11 19:21:00,2021-06-11,DXC,Dxc Technology Co,Fernandez Raul J,Dir,P - Purchase,41.20,1820.0,31305.0,6.0,74975.0,NaN,NaN,NaN,NaN
2,NaN,2021-06-11 16:50:00,2021-06-11,REVG,"Rev Group, Inc.",Boettinger Stephen W.,GC,P - Purchase,15.64,1500.0,121560.0,1.0,23460.0,NaN,NaN,NaN,NaN
3,NaN,2021-06-11 19:43:00,2021-06-11,AMST,Amesite Inc.,Sastry Ann Marie,CEO,P - Purchase,3.73,15000.0,6241667.0,0.0,55950.0,NaN,NaN,NaN,NaN
4,NaN,2021-06-11 19:47:00,2021-06-11,TLYS,"Tilly'S, Inc.",Zeichner Bernard,Dir,S - Sale,16.24,-5000.0,46894.0,-10.0,81200.0,NaN,NaN,NaN,NaN


In [86]:
len(insiderDat_filtered.Ticker.unique())

898

In [87]:
insiderDat_filtered.to_csv('data/insiderDat_June_clean.csv', index=False)